In [1]:
import datasets
from transformers import AutoTokenizer

In [17]:
dataset = datasets.load_dataset("openai/gsm8k", "main")
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")
tokenizer.pad_token = tokenizer.eos_token

In [3]:
q, a = dataset["train"][0]["question"], dataset["train"][0]["answer"]

In [18]:
q_tokens = tokenizer(q, return_tensors="pt")
qna_tokens = tokenizer(q + " Answer : " + a, return_tensors="pt", padding="max_length", max_length=128, truncation=True)

In [19]:
qna_tokens

{'input_ids': tensor([[128000,     45,   4306,    689,   6216,  27203,    311,    220,   2166,
            315,   1077,   4885,    304,   5936,     11,    323,   1243,   1364,
           6216,   4376,    439,   1690,  27203,    304,   3297,     13,   2650,
           1690,  27203,   1550,  42701,    689,   4662,  31155,    304,   5936,
            323,   3297,     30,  22559,    551,  42701,    689,   6216,    220,
           2166,     14,     17,    284,   1134,   2166,     14,     17,     28,
           1187,   2511,   1187,  27203,    304,   3297,    627,     45,   4306,
            689,   6216,    220,   2166,     10,   1187,    284,   1134,   2166,
             10,   1187,     28,   5332,   2511,   5332,  27203,  31155,    304,
           5936,    323,   3297,    627,    827,    220,   5332, 128001, 128001,
         128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001,
         128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001,
         12800

In [24]:
def test_func(exp):
    # Concatenate Question and Answer
    text = exp["question"] + " " + exp["answer"]
    # Tokenize the text
    tokenized = tokenizer(text, return_tensors="pt")
    # Get the attention mask
    attn_mask = tokenized["attention_mask"]
    # Get the length of the input
    inp_len = attn_mask.sum()
    
    
    # Attn mask for only the future token of the answer
res = dataset.map(test_func, batched=False)

Map:   0%|          | 0/7473 [00:00<?, ? examples/s]

Map:   0%|          | 0/1319 [00:00<?, ? examples/s]

In [27]:
res['train']['x'][0]

'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May? Natalia sold 48/2 = <<48/2=24>>24 clips in May.\nNatalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.\n#### 72'

In [20]:
res['train']['question'][0]

'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?'

In [21]:
res['train']['x'][0]

'a'

In [3]:
tokenizer.pad_token = tokenizer.eos_token

In [4]:
res = []
for i in range(len(dataset["train"])):
    q = dataset["train"][i]['question']
    a = dataset["train"][i]['answer']
    res.append(q + " " + a)
    if i > 2:
        break

In [9]:
token_res = tokenizer(res, padding="max_length", max_length=512, truncation=True)

In [14]:
len(token_res.input_ids), len(token_res.input_ids[0])

(4, 512)

In [5]:
tokenizer.pad_token = tokenizer.eos_token

# Function to concatenate and tokenize input
def preprocess_function(examples):
    # Concatenate question and answer
    inputs = [q + " " + a for q, a in zip(examples['question'], examples['answer'])]
    
    # Tokenize the input sequences
    tokenized_inputs = tokenizer(inputs, padding='max_length', truncation=True, return_tensors="pt")
    
    # Create attention masks: the answer portion should have causal masking
    attention_mask = tokenized_inputs.attention_mask.clone()  # Base attention mask
    
    for i, input_ids in enumerate(tokenized_inputs.input_ids):
        # Identify where the answer begins (we assume a simple space separation)
        q_len = len(tokenizer.tokenize(examples['question'][i]))
        a_len = len(tokenizer.tokenize(examples['answer'][i]))
        
        # Set attention masking so that tokens in the answer part only attend to previous answer tokens
        # Tokens before the answer can attend freely
        for j in range(q_len, q_len + a_len):
            attention_mask[i, j+1:] = 0  # Mask future answer tokens

    # Return tokenized inputs with attention masks
    return {
        "input_ids": tokenized_inputs.input_ids,
        "attention_mask": attention_mask
    }

# Apply the preprocessing function to the dataset
tokenized_dataset = dataset.map(preprocess_function, batched=True)

# Save the tokenized dataset
tokenized_dataset.save_to_disk("gsm8k_tokenized")
 


Map:   0%|          | 0/7473 [00:00<?, ? examples/s]

Map:   0%|          | 0/1319 [00:00<?, ? examples/s]

Saving the dataset (0/10 shards):   0%|          | 0/7473 [00:00<?, ? examples/s]

Saving the dataset (0/2 shards):   0%|          | 0/1319 [00:00<?, ? examples/s]

In [ ]:
STOP

In [2]:
# Load the tokenized dataset
tokenized_dataset = datasets.load_from_disk("gsm8k_tokenized")

In [3]:
tokenized_dataset['train']['input_ids'][0][0]